# 영화진흥위원회 데이터 수집

## setting

In [2]:
import requests

class URLMaker:
    """
    영화진흥위원회 API URL 생성 클래스
    """
    
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest'

    def __init__(self, key):
        """
        API key 초기화
        """
        self.key = key

    def daily_boxoffice_url(self, targetDt, itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        일별 박스오피스 URL 생성
        """
        return f'{self.url}/boxoffice/searchDailyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'

    def weekly_boxoffice_url(self, targetDt, weekGb='0', itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        주간/주말 박스오피스 URL 생성
        weekGb: 0(주간), 1(주말), 2(주중)
        """
        return f'{self.url}/boxoffice/searchWeeklyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&weekGb={weekGb}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'

    def movie_list_url(self, curPage=1, itemPerPage=10, movieNm='', directorNm='', openStartDt='', openEndDt='', 
                      prdtStartYear='', prdtEndYear='', repNationCd='', movieTypeCd=''):
        """
        영화목록 URL 생성
        """
        return f'{self.url}/movie/searchMovieList.json?key={self.key}&curPage={curPage}&itemPerPage={itemPerPage}&movieNm={movieNm}&directorNm={directorNm}&openStartDt={openStartDt}&openEndDt={openEndDt}&prdtStartYear={prdtStartYear}&prdtEndYear={prdtEndYear}&repNationCd={repNationCd}&movieTypeCd={movieTypeCd}'

    def movie_info_url(self, movieCd):
        """
        영화 상세정보 URL 생성
        """
        return f'{self.url}/movie/searchMovieInfo.json?key={self.key}&movieCd={movieCd}'

    def people_list_url(self, curPage=1, itemPerPage=10, peopleNm='', filmoNames=''):
        """
        영화인목록 URL 생성
        """
        return f'{self.url}/people/searchPeopleList.json?key={self.key}&curPage={curPage}&itemPerPage={itemPerPage}&peopleNm={peopleNm}&filmoNames={filmoNames}'

    def people_info_url(self, peopleCd):
        """
        영화인 상세정보 URL 생성
        """
        return f'{self.url}/people/searchPeopleInfo.json?key={self.key}&peopleCd={peopleCd}'


    

In [3]:
MY_API_KEY = 'ea85d0bf80c1a2a847737cf8270b6feb'
url_maker = URLMaker(MY_API_KEY)

In [4]:
url_maker.key

'ea85d0bf80c1a2a847737cf8270b6feb'

In [5]:
daily_url = url_maker.daily_boxoffice_url('20250201')
print(daily_url)

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=ea85d0bf80c1a2a847737cf8270b6feb&targetDt=20250201&itemPerPage=10&multiMovieYn=&repNationCd=&wideAreaCd=


In [18]:
movie_info_url = url_maker.movie_info_url('19950084')
print(movie_info_url)
# 이 구문이 함수 안으로 들어갔어야 했다
# '19950084'부분은 함수안으로 들어가면서 변수화 되어야 하겠다

http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json?key=ea85d0bf80c1a2a847737cf8270b6feb&movieCd=19950084


## 영화 상세정보 조회

- **영화 상세정보 api**를 사용합니다.
- 영화 코드(movieCd)를 입력받아 해당 영화의 상세 정보를 조회합니다.
- 다음 정보를 순서대로 출력합니다:
    - 영화 제목 (제작연도)
    - 감독 (첫 번째 감독)
    - 주연 배우 (첫 번째, 두 번째 배우)
    - 장르 (첫 번째 장르)

In [12]:
# movieNm
# directors
# actors_peopleNm[0, 1]
# genreNm

In [13]:
def get_movie_detail(movie_cd):
    r = requests.get(movie_info_url)
    print(r.json()['movieInfoResult']['movieInfo']['movieNm'], r.json()['movieInfoResult']['movieInfo']['prdtYear'])
    print(f'감독 : {r.json()['movieInfoResult']['movieInfo']['directors'][0]['peopleNm']}')
    print(f'주연 : {r.json()['movieInfoResult']['movieInfo']['actors'][0]['peopleNm']}, {r.json()['movieInfoResult']['movieInfo']['actors'][1]['peopleNm']}')
    print(f'감독 : {r.json()['movieInfoResult']['movieInfo']['genres'][0]['genreNm']}')

In [15]:
get_movie_detail('19950081')

쇼생크 탈출 1994
감독 : 프랭크 다라본트
주연 : 팀 로빈스, 모건 프리먼
감독 : 드라마


**출력 예시**
```
쇼생크 탈출 (1994)
감독: 프랭크 다라본트
주연: 팀 로빈스, 모건 프리먼
장르: 드라마
```

## 일별 박스오피스 조회

- **일별 박스오피스 api**를 사용합니다.
- 입력받은 날짜(YYYYMMDD 형식)의 박스오피스 정보를 조회합니다.
- 각 영화의 순위, 영화제목, 해당 일의 관객수를 출력합니다.

In [10]:
print(daily_url)
r = requests.get(daily_url)
daily_rank = r.json()['boxOfficeResult']['dailyBoxOfficeList']
print(daily_rank)
print(f'{daily_rank[0]['rank']}위 : {daily_rank[0]['movieNm']} ({daily_rank[0]['audiCnt']}명)')
# print(f'주연 : {r.json()['boxOfficeResult']['dailyBoxOfficeList']['actors'][0]['peopleNm']}')
# print(f'감독 : {r.json()['boxOfficeResult']['dailyBoxOfficeList']['genres'][0]['genreNm']}')

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=ea85d0bf80c1a2a847737cf8270b6feb&targetDt=20250201&itemPerPage=10&multiMovieYn=&repNationCd=K&wideAreaCd=
[{'rnum': '1', 'rank': '1', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20233006', 'movieNm': '히트맨2', 'openDt': '2025-01-22', 'salesAmt': '1663433100', 'salesShare': '46.0', 'salesInten': '525167660', 'salesChange': '46.1', 'salesAcc': '17240257840', 'audiCnt': '177129', 'audiInten': '40686', 'audiChange': '29.8', 'audiAcc': '1831557', 'scrnCnt': '1495', 'showCnt': '6165'}, {'rnum': '2', 'rank': '2', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20247450', 'movieNm': '검은 수녀들', 'openDt': '2025-01-24', 'salesAmt': '934872220', 'salesShare': '25.8', 'salesInten': '247202440', 'salesChange': '35.9', 'salesAcc': '13206216740', 'audiCnt': '96043', 'audiInten': '14734', 'audiChange': '18.1', 'audiAcc': '1363840', 'scrnCnt': '1192', 'showCnt': '4387'}, {'rnum': '3', 'rank': '3'

In [11]:
for movie in daily_rank:
    print(f'{movie['rank']}위 : {movie['movieNm']} ({movie['audiCnt']}명)')

1위 : 히트맨2 (177129명)
2위 : 검은 수녀들 (96043명)
3위 : 말할 수 없는 비밀 (63000명)
4위 : 하얼빈 (23298명)
5위 : 뽀로로 극장판 바닷속 대모험 (5903명)
6위 : 아이유 콘서트 : 더 위닝 (3156명)
7위 : 귀신경찰 (1860명)
8위 : 소방관 (1418명)
9위 : 마당을 나온 암탉 (1038명)
10위 : 뜻밖의 순간 : 언익스펙티드 저니 (241명)


In [19]:
def get_daily_boxoffice(target_date):
    daily_url = url_maker.daily_boxoffice_url(target_date)
    r = requests.get(daily_url)
    daily_rank = r.json()['boxOfficeResult']['dailyBoxOfficeList']
    
    for movie in daily_rank:
        print(f'{movie['rank']}위 : {movie['movieNm']} ({movie['audiCnt']}명)')

In [20]:
get_daily_boxoffice("20001123")  # YYYYMMDD 형식으로 날짜 입력

**출력예시**
```
20250201 박스오피스 TOP 10
1위: 히트맨2 (177129명)
2위: 검은 수녀들 (96043명)
3위: 말할 수 없는 비밀 (63000명)
4위: 하얼빈 (23298명)
5위: 서브스턴스 (15031명)
6위: 극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오 (12342명)
7위: 뽀로로 극장판 바닷속 대모험 (5903명)
8위: 바다 탐험대 옥토넛 어보브 앤 비욘드 : 극지방 대작전 (4570명)
9위: 극장판 짱구는 못말려: 우리들의 공룡일기 (3801명)
10위: 아이유 콘서트 : 더 위닝 (3156명)
```

## 한국 영화 일별 박스오피스 조회

- **일별 박스오피스 api**를 사용합니다.
- 입력받은 날짜(YYYYMMDD 형식)의 한국 영화 박스오피스 정보를 조회합니다.
- 한국 영화만 조회하기 위해 repNationCd='K' 파라미터를 사용합니다.
- 영화가 개봉한 순서대로 정렬하여 제목, 날짜, 순위를 출력합니다.


In [59]:
Kdaily_url = url_maker.daily_boxoffice_url('20250201', repNationCd='K')

print(Kdaily_url)

http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=ea85d0bf80c1a2a847737cf8270b6feb&targetDt=20250201&itemPerPage=10&multiMovieYn=&repNationCd=K&wideAreaCd=


In [115]:
from datetime import datetime

basic_daily = r.json()['boxOfficeResult']['dailyBoxOfficeList'][0]
open_dts = basic_daily['openDt'], basic_daily['movieNm'], basic_daily['rank']
open_dt = r.json()['boxOfficeResult']['dailyBoxOfficeList']
result = []

for i in open_dt:
    result.append([i['movieNm'], i['openDt'], i['rank']])

def get_date(result):
    if result[1].strip() == "":
        return datetime.min
    return datetime.strptime(result[1], '%Y-%m-%d')
result_sorted = sorted(result, key=get_date)

for j in result_sorted:
    print(j)

['뜻밖의 순간 : 언익스펙티드 저니', ' ', '10']
['마당을 나온 암탉', '2011-07-28', '9']
['소방관', '2024-12-04', '8']
['하얼빈', '2024-12-24', '4']
['뽀로로 극장판 바닷속 대모험', '2025-01-01', '5']
['히트맨2', '2025-01-22', '1']
['검은 수녀들', '2025-01-24', '2']
['아이유 콘서트 : 더 위닝', '2025-01-24', '6']
['귀신경찰', '2025-01-24', '7']
['말할 수 없는 비밀', '2025-01-27', '3']


In [57]:
def get_movie_details(target_date):
    daily_url = url_maker.daily_kboxoffice_url(target_date)
    r = requests.get(daily_url)

In [58]:
get_movie_details("20250201")  # YYYYMMDD 형식으로 날짜 입력

AttributeError: 'URLMaker' object has no attribute 'daily_kboxoffice_url'

**출력예시**
```
20250201 한국 영화 박스오피스 (개봉일 순)
뜻밖의 순간 : 언익스펙티드 저니 ( ) - 10위
마당을 나온 암탉 (2011-07-28) - 9위
소방관 (2024-12-04) - 8위
하얼빈 (2024-12-24) - 4위
뽀로로 극장판 바닷속 대모험 (2025-01-01) - 5위
히트맨2 (2025-01-22) - 1위
검은 수녀들 (2025-01-24) - 2위
아이유 콘서트 : 더 위닝 (2025-01-24) - 6위
귀신경찰 (2025-01-24) - 7위
말할 수 없는 비밀 (2025-01-27) - 3위
```

## 1월 전체 주간 박스오피스 데이터 수집

- **주간/주말 박스오피스 api**를 사용합니다.
- 2025년 1월의 모든 주간(1/5, 1/12, 1/19, 1/26) 데이터를 수집합니다.
- 각 영화의 정보는 다음을 포함해야 합니다:
    - 영화제목, 개봉일, 누적관객수, 누적매출액
- 동일한 영화가 여러 주에 걸쳐 나타날 경우, 가장 최근의 정보만 저장합니다.
```python
 def weekly_boxoffice_url(self, targetDt, weekGb='0', itemPerPage=10, multiMovieYn='', repNationCd='', wideAreaCd=''):
        """
        주간/주말 박스오피스 URL 생성
        weekGb: 0(주간), 1(주말), 2(주중)
        """
        return f'{self.url}/boxoffice/searchWeeklyBoxOfficeList.json?key={self.key}&targetDt={targetDt}&weekGb={weekGb}&itemPerPage={itemPerPage}&multiMovieYn={multiMovieYn}&repNationCd={repNationCd}&wideAreaCd={wideAreaCd}'
```


In [5]:
def get_weekly_boxoffice():
    jan = ['20250105', '20250112', '20250119', '20250126']
    for target_date in jan:
        pass
    pass

In [ ]:
get_weekly_boxoffice()

**출력예시**
```
{'하얼빈': {'영화제목': '하얼빈', '개봉일': '2024-12-24', '누적관객': '4642215', '누적매출': '44407362460'}, '소방관': {'영화제목': '소방관', '개봉일': '2024-12-04', '누적관객': '3840551', '누적매출': '36145572270'}, '보고타: 마지막 기회의 땅': {'영화제목': '보고타: 마지막 기회의 땅', '개봉일': '2024-12-31', '누적관객': '396287', '누적매출': '3669720540'}, '수퍼 소닉3': {'영화제목': '수퍼 소닉3', '개봉일': '2025-01-01', '누적관객': '322100', '누적매출': '2950620910'}, '뽀로로 극장판 바닷속 대모험': {'영화제목': '뽀로로 극장판 바닷속 대모험', '개봉일': '2025-01-01', '누적관객': '349597', '누적매출': '2996015770'}, '무파사: 라이온 킹': {'영화제목': '무파사: 라이온 킹', '개봉일': '2024-12-18', '누적관객': '833832', '누적매출': '8188072950'}, '모아나 2': {'영화제목': '모아나 2', '개봉일': '2024-11-27', '누적관객': '3522517', '누적매출': '32533317750'}, '극장판 짱구는 못말려: 우리들의 공룡일기': {'영화제목': '극장판 짱구는 못말려: 우리들의 공룡일기', '개봉일': '2024-12-18', '누적관객': '762895', '누적매출': '7177275100'}, '위키드': {'영화제목': '위키드', '개봉일': '2024-11-20', '누적관객': '2195165', '누적매출': '21677201940'}, '시빌 워: 분열의 시대': {'영화제목': '시빌 워: 분열의 시대', '개봉일': '2024-12-31', '누적관객': '80373', '누적매출': '836845230'}, '동화지만 청불입니다': {'영화제목': '동화지만 청불입니다', '개봉일': '2025-01-08', '누적관객': '157616', '누적매출': '1465916720'}, '서브스턴스': {'영화제목': '서브스턴스', '개봉일': '2024-12-11', '누적관객': '302870', '누적매출': '2954609400'}, '검은 수녀들': {'영화제목': '검은 수녀들', '개봉일': '2025-01-24', '누적관객': '606145', '누적매출': '5870106460'}, '히트맨2': {'영화제목': '히트맨2', '개봉일': '2025-01-22', '누적관객': '604875', '누적매출': '5686993120'}, '극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오': {'영화제목': '극장판 포켓몬스터 AG: 뮤와 파동의 용사 루카리오', '개봉일': '2025-01-22', '누적관객': '64585', '누적매출': '570033160'}, '아이유 콘서트 : 더 위닝': {'영화제목': '아이유 콘서트 : 더 위닝', '개봉일': '2025-01-24', '누적관객': '28061', '누적매출': '778067000'}, '귀신경찰': {'영화제목': '귀신경찰', '개봉일': '2025-01-24', '누적관객': '21949', '누적매출': '191035980'}}
```

## 일별 박스오피스 영화의 상세정보

- **일별 박스오피스 api**와 **영화 상세정보 api**를 사용합니다.
- 입력받은 날짜의 일별 박스오피스 TOP 10을 조회합니다.
- 각 영화의 상세 정보를 추가로 조회하여 다음 정보를 출력합니다:
    - 순위
    - 영화제목
    - 감독
    - 주연배우 (최대 3명)

In [ ]:
def get_movie_details(target_date):
    pass

In [ ]:
get_movie_details("20250201")  # YYYYMMDD 형식으로 날짜 입력

**출력예시**

```
1위
히트맨2
감독: 최원섭
주연: 권상우, 정준호, 이이경
----------------------------------------

2위
검은 수녀들
감독: 권혁재
주연: 송혜교, 전여빈, 이진욱
----------------------------------------

3위
말할 수 없는 비밀
감독: 서유민
주연: 도경수, 원진아, 신예은
----------------------------------------
...
```

## 배우검색

- **영화인목록 api**, **영화인 상세정보 api**, **영화 상세정보 api**를 사용합니다.
- 배우 이름을 입력받아 해당 배우의 정보를 검색합니다.
- 여러 명의 동명 배우가 있는 경우, 필모그래피가 가장 많은 배우를 선택합니다.
- 선택된 배우의 출연작 중 '배우'로 참여한 작품 5개의 정보를 출력합니다:
    - 영화 제목
    - 개봉일
    - 상영시간
    - 감독

In [ ]:
def analyze_actor_movies(actor_name):
    pass

In [ ]:
analyze_actor_movies("박정민")  # 원하는 배우 이름으로 변경 가능

**출력예시**

```
배우 '박정민'의 최근 출연작 5개

제목: 하얼빈
개봉일: 20241224
상영시간: 113분
감독: 우민호

제목: 얼굴
개봉일: 
상영시간: 분
감독: 연상호

제목: 전, 란
개봉일: 
상영시간: 126분
감독: 김상만

제목: 휴민트
개봉일: 
상영시간: 분
감독: 류승완

제목: 밀수
개봉일: 20230726
상영시간: 129분
감독: 류승완
```